In [8]:
import random
import logging # allows writing status messages to a file or output streams

import numpy as np 
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)
# set random seed
tf.keras.utils.set_random_seed(42)


# tf.keras.utils.set_random_seed?

In [ ]:
# Variable definition
MAX_DURATION = 1 # duration of the input audio file we feed into wav2vec
SAMPLING_RATE = 16000 # no. of samples of audio recorded every second
BATCH_SIZE = 32 # batch size for training and evaluating the model
NUM_CLASSES = 10 # classes our dataset will have
HIDDEN_DIM = 768 # Dimension of our model output (768 is for wav2vec2-base)
